In [19]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import hashlib

from numpy import cos, sin, pi
from numpy.random import randint as r_int

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout, GaussianNoise, GaussianDropout, LeakyReLU

from PIL import Image as IM, ImageDraw as ID, ImageFilter as IF

from IPython.display import display, Image

In [20]:
# Generates the Transformation Matrix for a Point
#   coord: The (x,y,z) point to transform
#   angle: The (a,b,c) angle to transform by
def shear_trans(coord=(0,0,0),angle=(0,0,0)):
    rot_x = np.array([[1,0,0],
                      [0,cos(angle[0]), sin(angle[0])],
                      [0,-sin(angle[0]),cos(angle[0])]],
                    dtype='float32')
    rot_y = np.array([[cos(angle[1]), 0, sin(angle[1])],
                      [0, 1, 0],
                      [-sin(angle[1]), 0, cos(angle[1])]],
                    dtype='float32')
    rot_z = np.array([[cos(angle[2]), -sin(angle[2]), 0],
                      [sin(angle[2]), cos(angle[2]), 0],
                      [0, 0, 1]],
                    dtype='float32')

    return np.dot(rot_x, rot_y, rot_z)

# Generates Points for Polygons of Specified Features
#   sides: Number of sides of polygon [default:3]
#   coord: Center of the polygon  
#   angle: Angle to rotate about center
#   scale: Value to magnify the polygon
#   shear: 3d Rotation to apply to polygon
def poly_gen(sides = 3,coord = (0,0), angle = 0, scale = 1, shear = (0,0,0)):
    p,x,y = [], *coord
    
    #Loop Through and Create All Verticies
    for s in range(sides):
        p.append( ( cos(s * 2 * pi / sides + angle) + x, sin(s * 2 * pi / sides + angle) + y))
        
    # Perform Shear Transform
    p = map(lambda q: np.dot(shear_trans((0,0,0), shear), np.array([*q,0]).T), p)
    p = list(map(lambda q: tuple(scale * (1 +  q.T[:-1] + np.array([x,y]))), p))
    
    return p

# Generates Data Set of Polygons
#   size: Number of Polygons to generate/Data Set Size
#   side_range: Range of sides
#   dim: Dimensions of the image
#   color: Color of the polygon
#   filled: Wether or not to fill in the polygon
#   output_dir: Whether or not to save images & where to save them
#   seed: Seed for numpy.random
def gen_poly_data(size = 1,side_range=(3,10), dim = (100,100), color=0, filled=True, blurred=False, output_dir = None, seed = 0):
    np.random.seed(seed)
    data = {'x':[],'y':[],'img':[]}
    few = 20
    # Generate Random Shapes
    for i in range(size):
        img = IM.new('L', dim,"#ffffff")
        draw = ID.Draw(img) 
        center = (1,1)#np.random.rand(2) * 3
        scale = np.random.randint(np.min(dim)/8, np.min(dim)/4)
        fill_color = color if filled else None
        num_sides = r_int(*side_range)

        draw.polygon(poly_gen(sides = num_sides,
                             coord = center,
                             angle = pi * np.random.rand(),
                             scale = scale,
                             shear = pi / 2 * np.random.rand(3) - pi/4),
                     fill = fill_color, outline = color)
        
        # Blur Image True
        if blurred:
            img = img.filter(IF.BLUR)
        
        data['img'].append(img)
        
        # Save Image to output folder if given
        if output_dir:
            img.save(os.path.join(output_dir,'img_{}_class_{}.jpg'.format(num_sides,i)))
        
        data['x'].append(np.array(img,dtype='float32').reshape(200,200,1)/255)
        
        # One Hot Encode Label
        one_hot = np.zeros(side_range[1] - side_range[0])
        one_hot[num_sides - side_range[0]] = 1
        data['y'].append(one_hot.T)
        
    return data



In [21]:
def init_model():
    model = Sequential()
    model.add(Conv2D(64, (4, 4), input_shape = (200, 200,1), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    # model.add(GaussianDropout(0.15))
    model.add(Conv2D(16, (4, 4), activation='relu'))
    # model.add(GaussianNoise(1))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(units = 64, activation = 'sigmoid'))
    model.add(Dense(units = 22, activation = 'softmax'))
    model.add(Dense(units = 7, activation = 'sigmoid'))
    
#     model.add(Conv2D(64, (4, 4), input_shape = (200, 200, 1), activation='relu'))
#     model.add(MaxPooling2D(pool_size = (8, 8)))
#     model.add(Flatten())
#     model.add(Dense(units = 64, activation = 'sigmoid'))
#     model.add(Dense(units = 4, activation = 'sigmoid'))
    
    model.summary()
    return model

In [22]:
train_size, test_size = 5000, 500# 10000, 500
train_set = gen_poly_data(train_size,side_range=(3,10), dim = (200,200), color=0, filled=True, blurred=False, output_dir = None, seed = 0)
test_set = gen_poly_data(test_size,side_range=(3,10), dim = (200,200), color=0, filled=True, blurred=False, output_dir = None, seed = 1)

hasher = hashlib.md5()
with open('auxiliary.ipynb', 'rb') as this_file:
    hasher.update(this_file.read())
this_hash = str(hasher.hexdigest())[:8]

epochs1, epochs2 = 8,8

In [23]:
try:
    with open('mean_squared_model-' + this_hash + '.h5', 'r') as fh:
        model1 = keras.models.load_model('mean_squared_model-' + this_hash + '.h5')
except FileNotFoundError:
    model1 = init_model()
    model1.compile(optimizer = tf.train.AdamOptimizer(), loss = 'mean_squared_error', metrics=['accuracy'])
    model1.fit(np.array(train_set['x']), np.array(train_set['y']), batch_size = 64, epochs = epochs1, validation_split=0.1)
    model1.save('mean_squared_model-' + this_hash + '.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 197, 197, 64)      1088      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 95, 95, 16)        16400     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 47, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 35344)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2262080   
_________________________________________________________________
dense_4 (Dense)              (None, 22)                1430      
__________

In [24]:
try:
    with open('cross_entropy_model-' + this_hash + '.h5', 'r') as fh:
        model2 = keras.models.load_model('cross_entropy_model-' + this_hash + '.h5')
except FileNotFoundError:
    model2 = init_model()
    model2.compile(optimizer = tf.train.AdamOptimizer(), loss = 'categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])
    model2.fit(np.array(train_set['x']), np.array(train_set['y']), batch_size = 32, epochs = epochs2, validation_split=0.1)
    model2.save('cross_entropy_model-' + this_hash + '.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 197, 197, 64)      1088      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 95, 95, 16)        16400     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 47, 47, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 35344)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                2262080   
_________________________________________________________________
dense_7 (Dense)              (None, 22)                1430      
__________

In [25]:
plt.plot(Out[5].history['acc'])
plt.plot(Out[5].history['loss'])
plt.plot(Out[5].history['val_acc'])
plt.plot(Out[5].history['val_loss'])
plt.show()

AttributeError: 'list' object has no attribute 'history'

In [ ]:
plt.plot(Out[6].history['acc'])
plt.plot(Out[6].history['loss'])
plt.plot(Out[6].history['categorical_accuracy'])
plt.plot(Out[6].history['val_acc'])
plt.plot(Out[6].history['val_loss'])
plt.plot(Out[6].history['val_categorical_accuracy'])
plt.show()

In [ ]:
correct = 0
for i in range(test_size):
    print('Checking Guess #{}:'.format(i),end='\n\t')
    p = model1.predict(test_set['x'][i].reshape(1,200,200,1))
    if np.argmax(p) == np.argmax(test_set['y'][i]):
        print('[\] - Match!')
        correct = correct + 1
    else:
        print('[X] - Miss')        
        display(test_set['img'][i])
    print('Sides:\n\tGuess:{}\n\tActual:{}'.format( 3 + np.argmax(p), 3 + np.argmax(test_set['y'][i])),'\n')

print('Results: {}%'.format(correct/test_size * 100))

In [ ]:
correct = 0
for i in range(test_size):
    print('Checking Guess #{}:'.format(i),end='\n\t')
    p = model2.predict(test_set['x'][i].reshape(1,200,200,1))
    if np.argmax(p) == np.argmax(test_set['y'][i]):
        print('[\] - Match!')
        correct = correct + 1
    else:
        print('[X] - Miss')        
        display(test_set['img'][i])
    print('Sides:\n\tGuess:{}\n\tActual:{}'.format( 3 + np.argmax(p), 3 + np.argmax(test_set['y'][i])),'\n')

print('Results: {}%'.format(correct/test_size * 100))

In [ ]:

dir(gen_poly_data)
# gen_poly_data.__getattribute__()
gen_poly_data.__defaults__